In [3]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

In [5]:
def data():
    # Split train & validation set
    DDD_Train_Decile10 = pd.read_pickle("DDD_Train_Decile10.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile10.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile10, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 10')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 13s 6ms/step - loss: 0.2856 - acc: 0.8874 - val_loss: 0.2131 - val_acc: 0.9206
Epoch 35/75
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2872 - acc: 0.8869 - val_loss: 0.2126 - val_acc: 0.9206
Epoch 36/75
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2863 - acc: 0.8816 - val_loss: 0.2129 - val_acc: 0.9206

Epoch 00036: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 37/75
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2818 - acc: 0.8852 - val_loss: 0.2128 - val_acc: 0.9206
Epoch 38/75
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2820 - acc: 0.8847 - val_loss: 0.2127 - val_acc: 0.9206
Epoch 39/75
2264/2264 [==============================] - 15s 6ms/step - loss: 0.2882 - acc: 0.8830 - val_loss: 0.2128 - val_acc: 0.9206
Epoch 40/75
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2734 - acc: 0.8944 - val_loss: 0.2128 - val_acc: 0.

2264/2264 [==============================] - 13s 6ms/step - loss: 0.2703 - acc: 0.8860 - val_loss: 0.2169 - val_acc: 0.9101
Epoch 18/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2661 - acc: 0.8896 - val_loss: 0.2162 - val_acc: 0.9118
Epoch 19/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2727 - acc: 0.8887 - val_loss: 0.2162 - val_acc: 0.9118

Epoch 00019: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 20/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2688 - acc: 0.8909 - val_loss: 0.2164 - val_acc: 0.9118
Epoch 21/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2634 - acc: 0.8887 - val_loss: 0.2168 - val_acc: 0.9118
Epoch 22/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2666 - acc: 0.8883 - val_loss: 0.2168 - val_acc: 0.9118
Epoch 23/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2673 - acc: 0.8874 - val_loss: 0.2169 - val_a

2264/2264 [==============================] - 13s 6ms/step - loss: 0.2761 - acc: 0.8891 - val_loss: 0.2164 - val_acc: 0.9101
Epoch 77/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2665 - acc: 0.8869 - val_loss: 0.2166 - val_acc: 0.9101
Epoch 78/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2650 - acc: 0.8896 - val_loss: 0.2166 - val_acc: 0.9101
Epoch 79/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2796 - acc: 0.8856 - val_loss: 0.2167 - val_acc: 0.9101
Epoch 80/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2675 - acc: 0.8900 - val_loss: 0.2166 - val_acc: 0.9101
Epoch 81/100
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2667 - acc: 0.8878 - val_loss: 0.2167 - val_acc: 0.9101
Epoch 82/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2638 - acc: 0.8931 - val_loss: 0.2166 - val_acc: 0.9101
Epoch 83/100
2264/2264 [==============================

Epoch 34/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2802 - acc: 0.8887 - val_loss: 0.2076 - val_acc: 0.9259
Epoch 35/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2956 - acc: 0.8830 - val_loss: 0.2076 - val_acc: 0.9259
Epoch 36/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2895 - acc: 0.8838 - val_loss: 0.2076 - val_acc: 0.9259
Epoch 37/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2959 - acc: 0.8803 - val_loss: 0.2074 - val_acc: 0.9259
Epoch 38/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2848 - acc: 0.8883 - val_loss: 0.2076 - val_acc: 0.9259
Epoch 39/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.2840 - acc: 0.8887 - val_loss: 0.2075 - val_acc: 0.9259
Epoch 40/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.2925 - acc: 0.8838 - val_loss: 0.2076 - val_acc: 0.9259
Epoch 41/100
2264/2264 [=================

...

2264/2264 [==============================] - 13s 6ms/step - loss: 0.5756 - acc: 0.7513 - val_loss: 0.5434 - val_acc: 0.7813
Epoch 6/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.5289 - acc: 0.7836 - val_loss: 0.4999 - val_acc: 0.7919
Epoch 7/50
2264/2264 [==============================] - 14s 6ms/step - loss: 0.5025 - acc: 0.7911 - val_loss: 0.4650 - val_acc: 0.8095
Epoch 8/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.4741 - acc: 0.8140 - val_loss: 0.4352 - val_acc: 0.8377
Epoch 9/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.4511 - acc: 0.8291 - val_loss: 0.4073 - val_acc: 0.8642
Epoch 10/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.4314 - acc: 0.8366 - val_loss: 0.3813 - val_acc: 0.8818
Epoch 11/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.4221 - acc: 0.8366 - val_loss: 0.3648 - val_acc: 0.8924
Epoch 12/50
2264/2264 [==============================] - 13s 6ms

Epoch 15/75
2264/2264 [==============================] - 14s 6ms/step - loss: 0.3373 - acc: 0.8750 - val_loss: 0.2410 - val_acc: 0.9136
Epoch 16/75
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3369 - acc: 0.8723 - val_loss: 0.2325 - val_acc: 0.9277
Epoch 17/75
2264/2264 [==============================] - 14s 6ms/step - loss: 0.3192 - acc: 0.8856 - val_loss: 0.2338 - val_acc: 0.9153
Epoch 18/75
2264/2264 [==============================] - 14s 6ms/step - loss: 0.3206 - acc: 0.8746 - val_loss: 0.2305 - val_acc: 0.9259
Epoch 19/75
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3098 - acc: 0.8830 - val_loss: 0.2278 - val_acc: 0.9295
Epoch 20/75
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3215 - acc: 0.8790 - val_loss: 0.2302 - val_acc: 0.9242
Epoch 21/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3094 - acc: 0.8794 - val_loss: 0.2191 - val_acc: 0.9206
Epoch 22/75
2264/2264 [=========================

2264/2264 [==============================] - 13s 6ms/step - loss: 0.2944 - acc: 0.8843 - val_loss: 0.2154 - val_acc: 0.9206
Epoch 75/75
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2955 - acc: 0.8869 - val_loss: 0.2157 - val_acc: 0.9206
Test accuracy: 0.920634919899
Train on 2264 samples, validate on 567 samples
Epoch 1/100
2264/2264 [==============================] - 20s 9ms/step - loss: 0.4784 - acc: 0.7902 - val_loss: 0.3164 - val_acc: 0.8854
Epoch 2/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3615 - acc: 0.8662 - val_loss: 0.2389 - val_acc: 0.9153
Epoch 3/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3533 - acc: 0.8626 - val_loss: 0.2401 - val_acc: 0.9118
Epoch 4/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3094 - acc: 0.8816 - val_loss: 0.2837 - val_acc: 0.8818
Epoch 5/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3118 - acc: 0.8741 - val_loss: 0.2221 -

Epoch 57/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2686 - acc: 0.8891 - val_loss: 0.2134 - val_acc: 0.9118
Epoch 58/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2632 - acc: 0.8922 - val_loss: 0.2134 - val_acc: 0.9118
Epoch 59/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2644 - acc: 0.8913 - val_loss: 0.2134 - val_acc: 0.9118
Epoch 60/100
2264/2264 [==============================] - 12s 6ms/step - loss: 0.2633 - acc: 0.8891 - val_loss: 0.2134 - val_acc: 0.9118
Epoch 61/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2653 - acc: 0.8865 - val_loss: 0.2134 - val_acc: 0.9118
Epoch 62/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2674 - acc: 0.8891 - val_loss: 0.2134 - val_acc: 0.9118
Epoch 63/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2620 - acc: 0.8918 - val_loss: 0.2135 - val_acc: 0.9118
Epoch 64/100
2264/2264 [=================

2264/2264 [==============================] - 13s 6ms/step - loss: 0.2864 - acc: 0.8847 - val_loss: 0.2088 - val_acc: 0.9224
Epoch 17/50
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2898 - acc: 0.8816 - val_loss: 0.2081 - val_acc: 0.9136
Epoch 18/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2795 - acc: 0.8843 - val_loss: 0.2102 - val_acc: 0.9206
Epoch 19/50
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2817 - acc: 0.8812 - val_loss: 0.2083 - val_acc: 0.9171
Epoch 20/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2729 - acc: 0.8900 - val_loss: 0.2134 - val_acc: 0.9153

Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 21/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.2725 - acc: 0.8905 - val_loss: 0.2122 - val_acc: 0.9153
Epoch 22/50
2264/2264 [==============================] - 14s 6ms/step - loss: 0.2711 - acc: 0.8909 - val_loss: 0.2

In [ ]:
0.86434